In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade caer canaro

     |████████████████████████████████| 809 kB 5.3 MB/s 
     |████████████████████████████████| 14.0 MB 40.4 MB/s 
     |████████████████████████████████| 843 kB 39.9 MB/s 


In [ ]:
import caer
import canaro
import numpy as np
import cv2 as cv
import gc

In [ ]:
IMG_SIZE= (80,80)
channels = 1
char_path ='/content/drive/MyDrive/simpsons_dataset'

In [ ]:
import os
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))
    
char_dict   = caer.sort_dict(char_dict, descending = True)
char_dict

[('homer_simpson', 2246),
 ('lisa_simpson', 1354),
 ('bart_simpson', 1342),
 ('marge_simpson', 1291),
 ('krusty_the_clown', 1206),
 ('charles_montgomery_burns', 1193),
 ('chief_wiggum', 986),
 ('abraham_grampa_simpson', 913),
 ('apu_nahasapeemapetilon', 623),
 ('kent_brockman', 498),
 ('comic_book_guy', 469),
 ('edna_krabappel', 457),
 ('lenny_leonard', 310),
 ('maggie_simpson', 128),
 ('groundskeeper_willie', 121),
 ('barney_gumble', 106),
 ('carl_carlson', 98),
 ('martin_prince', 71),
 ('cletus_spuckler', 47),
 ('agnes_skinner', 42),
 ('gil', 27),
 ('fat_tony', 27),
 ('disco_stu', 8),
 ('lionel_hutz', 3)]

In [ ]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count+=1
    if count>=10:
        break
characters

['homer_simpson',
 'lisa_simpson',
 'bart_simpson',
 'marge_simpson',
 'krusty_the_clown',
 'charles_montgomery_burns',
 'chief_wiggum',
 'abraham_grampa_simpson',
 'apu_nahasapeemapetilon',
 'kent_brockman']

In [ ]:
train = caer.preprocess_from_dir(char_path, characters, channels = channels, IMG_SIZE = IMG_SIZE, isShuffle = True, verbose = True)


[INFO] Could not find a file to load from. Generating the training data
----------------------------------------------
[INFO] At 1000 files
[INFO] At 2000 files
[INFO] 2246 files found in 0.11093688011169434s
[INFO] At 1000 files
[INFO] 1354 files found in 0.024522781372070312s
[INFO] At 1000 files
[INFO] 1342 files found in 0.023793935775756836s
[INFO] At 1000 files
[INFO] 1291 files found in 0.033728837966918945s
[INFO] At 1000 files
[INFO] 1206 files found in 0.021969318389892578s
[INFO] At 1000 files
[INFO] 1193 files found in 0.020994186401367188s
[INFO] 986 files found in 0.016427993774414062s
[INFO] 913 files found in 0.015804529190063477s
[INFO] 623 files found in 0.010954141616821289s
[INFO] 498 files found in 0.010161876678466797s
----------------------------------------------
[INFO] 11652 files preprocessed! Took 3m 56s


In [ ]:
featureSet , labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
#normalize the feautureSet ==> (0,1)

from tensorflow.keras.utils import to_categorical

featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [ ]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)

In [ ]:
del train
del featureSet
del labels
gc.collect()

131

In [ ]:
BATCH_SIZE  = 4
EPOCHS = 15

In [ ]:
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train,y_train, batch_size = BATCH_SIZE)

In [ ]:
#creating a model

# Create our model (returns a compiled model)
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels, output_dim=len(characters), 
                                         loss='binary_crossentropy', decay=1e-7, learning_rate=0.002, momentum=0.9,
                                         nesterov=True)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 80, 80, 32)        320       
                                                                 
 conv2d_7 (Conv2D)           (None, 78, 78, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 39, 39, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 39, 39, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 39, 39, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 37, 37, 64)        36928     
                                                      

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
#train

from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]
training = model.fit(train_gen,
                    steps_per_epoch=int(len(x_train)/BATCH_SIZE),
                    epochs=EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps=int(len(y_val)/BATCH_SIZE),
                    callbacks = callbacks_list)

Epoch 1/15
2330/2330 [==============================] - 69s 26ms/step - loss: 0.3195 - accuracy: 0.1837 - val_loss: 0.2986 - val_accuracy: 0.2762 - lr: 0.0100
Epoch 2/15
2330/2330 [==============================] - 58s 25ms/step - loss: 0.2941 - accuracy: 0.2602 - val_loss: 0.2767 - val_accuracy: 0.3255 - lr: 0.0100
Epoch 3/15
2330/2330 [==============================] - 59s 25ms/step - loss: 0.2802 - accuracy: 0.3202 - val_loss: 0.2503 - val_accuracy: 0.4511 - lr: 0.0100
Epoch 4/15
2330/2330 [==============================] - 59s 25ms/step - loss: 0.2635 - accuracy: 0.3785 - val_loss: 0.2461 - val_accuracy: 0.4524 - lr: 0.0100
Epoch 5/15
2330/2330 [==============================] - 57s 24ms/step - loss: 0.2511 - accuracy: 0.4165 - val_loss: 0.2292 - val_accuracy: 0.4833 - lr: 0.0100
Epoch 6/15
2330/2330 [==============================] - 59s 25ms/step - loss: 0.2360 - accuracy: 0.4649 - val_loss: 0.2045 - val_accuracy: 0.5682 - lr: 0.0100
Epoch 7/15
2330/2330 [========================